O artigo do Medium: "How to Build a Vegan Itinerary Suggester Using LangChain, Google Maps API, and Gradio" explora a criação de um sistema para busca e geração de itinerários de restaurantes veganos, integrando as tecnologias LangChain, API Google Maps, Gradio e OpenAI.

**LangChain:** É uma ferramenta que facilita a criação de agentes de linguagem natural capazes de interagir com várias APIs e sistemas externos, como a OpenAI, utilizada neste projeto.

**Google Maps API:** Fornece dados geográficos e de localização, permitindo que o sistema encontre e sugira restaurantes veganos em uma determinada área. A API permite obter coordenadas precisas, listar restaurantes próximos e calcular rotas, facilitando a criação de itinerários detalhados que incluem os melhores pontos de interesse. Além disso, é utilizada para obter informações atualizadas sobre os locais, como endereço e horário de funcionamento.

**Gradio:** É uma ferramenta que permite a criação de interfaces de usuário interativas de maneira rápida e intuitiva. Neste projeto, Gradio é utilizado para desenvolver a interface web onde os usuários podem inserir suas preferências, escolher a cidade de destino e visualizar o itinerário sugerido.


Artigo: https://medium.com/@lorenamelo.engr/how-to-create-an-itinerary-suggester-using-langchain-google-maps-api-and-gradio-7c281bd54d2f

In [ ]:
# Etapa 1: Configurando o Ambiente
!pip install gradio googlemaps folium langchain
!pip install -qU langchain-openai

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 58.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 38.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 395.9/395.9 kB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.6/150.6 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.3/10.3 MB 58.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.5/93.5 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━

In [ ]:
# Etapa 2: configurar chaves de API e o modelo LLM

from langchain_openai import ChatOpenAI
from getpass import getpass

#API key GOOGLE Maps
open_ai_api_key = getpass()
google_maps_api_key = getpass()

# Initialize the LangChain model
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0, openai_api_key=open_ai_api_key)

··········
··········


In [ ]:
# Etapa 3: Configurando funções para buscar dados

from typing import List, Dict
from langchain.agents import tool
import googlemaps

# Função para obter coordenadas de uma cidade
def get_coordinates(google_maps_api_key, city_name):
    gmaps = googlemaps.Client(key=google_maps_api_key)
    geocode_result = gmaps.geocode(city_name)
    if not geocode_result:
        raise ValueError(f"City name '{city_name}' not found.")
    # Extrai a latitude e longitude da resposta
    location = geocode_result[0]['geometry']['location']
    return (location['lat'], location['lng'])

# Função para buscar restaurantes veganos na cidade solicitada
def get_vegan_restaurants(google_maps_api_key, city_name, radius=5000):
    lat, lng = get_coordinates(google_maps_api_key, city_name)
    gmaps = googlemaps.Client(key=google_maps_api_key)
    # Busca por restaurantes veganos proximos usando as coordenadas e o raio
    places_result = gmaps.places_nearby(location=(lat, lng), radius=radius, type='restaurant', keyword='vegan')
    return places_result['results']

def get_restaurants_by_kind(google_maps_api_key, city_name, kind, radius=5000):
    lat, lng = get_coordinates(google_maps_api_key, city_name)
    gmaps = googlemaps.Client(key=google_maps_api_key)
    # Busca por restaurantes veganos proximos usando as coordenadas e o raio
    places_result = gmaps.places_nearby(location=(lat, lng), radius=radius, type='restaurant', keyword=kind)
    return places_result['results']

In [ ]:
# Exemplo de resultado

location = "São Paulo"  # enter a city name
vegan_restaurants = get_vegan_restaurants(google_maps_api_key, location)

# Print the results
for restaurant in vegan_restaurants:
    print(f"{restaurant['name']} - {restaurant['vicinity']}")

Casa RAW - R. Dr. Franco da Rocha, 515 - Perdizes, São Paulo
Restaurante Taste and See (vegano) - Alameda Campinas, 234 - Jardim Paulista, São Paulo
Dona Augusta - R. Augusta, 1112 - Jardim America, São Paulo
Lar Vegan Restaurante, Pizzaria e Espaço Cultural - R. Clélia, 284 - Água Branca, São Paulo
Loving Hut Vila Mariana - R. França Pinto, 243 - Vila Mariana, São Paulo
Novos Veganos - Unidade Pinheiros - R. Fidalga, 73 - Pinheiros, São Paulo
Lagoa Tropical - R. Borges Lagoa, 406 - Vila Clementino, São Paulo
Banana Verde Restaurant - Rua Harmonia, 278 - Vila Madalena, São Paulo
Vita Natural - Alameda Campinas, 244 - Jardim Paulista, São Paulo
Pop Vegan Food - R. Fernando de Albuquerque, 142/144 - Consolação, São Paulo
It's Vegan - R. Fernando de Albuquerque, 89 - Consolação, São Paulo
Alternativa | Casa de Conexão | Vila Madalena - R. Fradique Coutinho, 910 - Vila Madalena, São Paulo
Theo's Vegan Burguer and Dogs - R. Conselheiro Ramalho, 693 - Bela Vista, São Paulo
CPV - Cozinha Popu

In [ ]:
# 4.1 Localizador de restaurantes veganos

@tool
def find_vegan_restaurants(city_name: str) -> List[Dict]:
    """
    Find vegan restaurants near the specified city.

    Args:
        city_name (str): The city to search for vegan restaurants.

    Returns:
        List[Dict]: A list of dictionaries containing restaurant information.
    """
    # Chama a função get_vegan_restaurants
    restaurants = get_vegan_restaurants(google_maps_api_key, city_name)

    # Retorna uma lista de dicionários com informações dos restaurantes encontrados
    return [
        {
            "name": restaurant["name"],
            "address": restaurant["vicinity"],
            "lat": restaurant["geometry"]["location"]["lat"],
            "lng": restaurant["geometry"]["location"]["lng"]
        }
        for restaurant in restaurants
    ]

@tool
def find_restaurants_by_kind(city_name: str, kind: str) -> List[Dict]:
    """
    Find restaurants of the specified kind near the specified city.

    Args:
        city_name (str): The city to search for restaurants of kind.
        kind (str): The kind of the restaurant to be searched.

    Returns:
        List[Dict]: A list of dictionaries containing restaurant information.
    """

    # Chama a função get_restaurants_by_kind
    restaurants = get_restaurants_by_kind(google_maps_api_key, city_name, kind)

    # Retorna uma lista de dicionários com informações dos restaurantes encontrados
    return [
        {
            "name": restaurant["name"],
            "address": restaurant["vicinity"],
            "lat": restaurant["geometry"]["location"]["lat"],
            "lng": restaurant["geometry"]["location"]["lng"]
        }
        for restaurant in restaurants
    ]


In [ ]:
# 4.2 Vincular ferramentas ao LLM

# Cria uma lista contendo a função find_vegan_restaurants como uma ferramenta
tools = [find_vegan_restaurants, find_restaurants_by_kind]

# Vincula as ferramentas ao LLM
llm_with_tools = llm.bind_tools(tools)

In [ ]:
# 4.3 Crie o prompt

from langchain_core.prompts import MessagesPlaceholder
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder


MEMORY_KEY = "chat_history"
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
             """You are a travel assistant specializing in helping users find the kind specified restaurants and plan activities
            in their travel destinations. You have access to a tool called find_restaurants_by_kind to help you find restaurants by kind.

            When a user asks for a kind of restaurants in a specific city, use the find_restaurants_by_kind tool to fetch and return a list
            of this kind of restaurants in that city.

            If the user asks for an itinerary, generate a detailed itinerary for the given number of days, including visits to this kind of
            restaurants and popular landmarks.

            Only use valid city names and trip durations. If the user provides invalid input, respond with an error message asking
            for the necessary details.

            A valid request should contain the following:
            - A city name
            - A trip duration that is reasonable for exploring restaurants and activities
            - Some other details, like the user's interests and/or specific requirements

            Any request that contains potentially harmful activities is not valid, regardless of what
            other details are provided.

            If the request is not valid, set plan_is_valid = 'no' and use your expertise to update the request to make it valid,
            keeping your revised request shorter than 100 words.

            If the request seems reasonable, then set plan_is_valid = 'yes' and
            don't revise the request.""",

        ),
        MessagesPlaceholder(variable_name=MEMORY_KEY),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)


# Adiciona o histórico de chat

from langchain_core.messages import AIMessage, HumanMessage

chat_history = []

In [ ]:
# 4.4 Crie o Agente

# Importa funções para formatar e processar mensagens com as ferramentas da OpenAI
from langchain.agents.format_scratchpad.openai_tools import (
    format_to_openai_tool_messages,
)
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser

# Configura o agente
agent = (
    {
        "input": lambda x: x["input"],
        # Formata as mensagens geradas pelo agente durante o processamento
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
        # Recupera o histórico de chat
        "chat_history": lambda x: x["chat_history"],
    }
    | prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)

#configura o agente e as ferramentas, e ativa o modo verbose para mais detalhes durante a execução
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [ ]:
input1 = "Which vegan places can I visit in São Paulo?"
result = agent_executor.invoke({"input": input1, "chat_history": chat_history})



> Entering new AgentExecutor chain...

Invoking: `find_vegan_restaurants` with `{'city_name': 'São Paulo'}`


[{'name': 'Lar Vegan Restaurante, Pizzaria e Espaço Cultural', 'address': 'R. Clélia, 284 - Água Branca, São Paulo', 'lat': -23.5248715, 'lng': -46.6852037}, {'name': "It's Vegan", 'address': 'R. Fernando de Albuquerque, 89 - Consolação, São Paulo', 'lat': -23.5544686, 'lng': -46.65751119999999}, {'name': 'Dona Augusta', 'address': 'R. Augusta, 1112 - Jardim America, São Paulo', 'lat': -23.5540884, 'lng': -46.6560693}, {'name': "Theo's Vegan Burguer and Dogs", 'address': 'R. Conselheiro Ramalho, 693 - Bela Vista, São Paulo', 'lat': -23.5579584, 'lng': -46.64425749999999}, {'name': 'Casa RAW', 'address': 'R. Dr. Franco da Rocha, 515 - Perdizes, São Paulo', 'lat': -23.5367831, 'lng': -46.6729673}, {'name': 'Banana Verde Restaurant', 'address': 'Rua Harmonia, 278 - Vila Madalena, São Paulo', 'lat': -23.5554785, 'lng': -46.687619}, {'name': 'Novos Veganos - Unidade Pinheiros', 'a

In [ ]:
input2 = "Which italian places can I visit in São Paulo?"
result = agent_executor.invoke({"input": input2, "chat_history": chat_history})



> Entering new AgentExecutor chain...

Invoking: `find_restaurants_by_kind` with `{'city_name': 'São Paulo', 'kind': 'Italian'}`


[{'name': 'Famiglia Mancini Trattoria', 'address': 'Rua Avanhandava, 81, Bela Vista - Centro Histórico de São Paulo, São Paulo', 'lat': -23.5503587, 'lng': -46.6450067}, {'name': 'Circolo Cucina', 'address': 'Av. Ipiranga, 344 - Centro Histórico de São Paulo, São Paulo', 'lat': -23.5455136, 'lng': -46.64351}, {'name': 'PASTA E POMODORI TRATTORIA', 'address': 'R. Cel. Domingos Ferreira, 23 - Complemento 01 - Cursino, São Paulo', 'lat': -23.5993964, 'lng': -46.6164128}, {'name': 'Sanremo Italian Food', 'address': 'R. Dr. Homem de Melo, 450 - Perdizes, São Paulo', 'lat': -23.5366136, 'lng': -46.6689592}, {'name': 'Enosteria Vino e Cucina', 'address': 'R. Oscar Freire, 574 - Cerqueira César, São Paulo', 'lat': -23.5650262, 'lng': -46.666687}, {'name': 'Trattoria Tavolino Jardins', 'address': 'Alameda Lorena, 558 - Jardim Paulista, São Paulo', 'lat': -23.56999

In [ ]:
# Verificando e usando chat_history:

chat_history.extend(
    [
        # Adiciona a mensagem do usuário ao histórico
        HumanMessage(content=input1),
        # Adiciona a resposta do agente ao histórico
        AIMessage(content=result["output"]),
    ]
)
result_itinerary = agent_executor.invoke({"input": "Make a intinerary for 5 days so I can visit the most of these restaurants and also some landmarks in the city. Answer in portuguese", "chat_history": chat_history})



> Entering new AgentExecutor chain...
**Itinerário de 5 Dias em São Paulo**

**Dia 1: Chegada e Exploração do Centro**
- Manhã: Chegada a São Paulo e check-in no hotel.
- Almoço: **Famiglia Mancini Trattoria** - Experimente pratos tradicionais italianos.
- Tarde: Visite o **Theatro Municipal** e a **Praça da Sé**.
- Jantar: **Circolo Cucina** - Saboreie uma deliciosa refeição italiana.

**Dia 2: Cultura e Gastronomia**
- Manhã: Visite o **Museu de Arte de São Paulo (MASP)**.
- Almoço: **Eataly** - Aproveite a variedade de opções italianas.
- Tarde: Passeio pelo **Parque do Ibirapuera**.
- Jantar: **Trattoria Tavolino Jardins** - Desfrute de um jantar aconchegante.

**Dia 3: Arte e História**
- Manhã: Visite o **Museu da Imigração**.
- Almoço: **Pasta e Pomodori Trattoria** - Experimente pratos de massa.
- Tarde: Explore o **Beco do Batman** e a **Vila Madalena**.
- Jantar: **Altruísta Osteria e Enoteca** - Uma experiência gastronômica única.

**Dia 4: Compras e Relaxamento**
- Manhã:

In [ ]:
import folium
from folium.plugins import MarkerCluster
#Etapa 5: Criando o Mapa com Folium

# Função para criar um mapa usando Folium
def create_map(locations: List[Dict[str, float]],draw_route: bool = False):
    if not locations:
        return "No locations to display on the map."
    map_center = [locations[0]['lat'], locations[0]['lng']]
    folium_map = folium.Map(location=map_center, zoom_start=13)
    marker_cluster = MarkerCluster().add_to(folium_map)

    for location in locations:
        folium.Marker(
            location=[location['lat'], location['lng']],
            popup=location['name']
        ).add_to(marker_cluster)

    # Desenhar rota se necessário
    if draw_route and len(locations) > 1:
        route = [(location['lat'], location['lng']) for location in locations]
        folium.PolyLine(route, color="blue", weight=2.5, opacity=1).add_to(folium_map)

    return folium_map._repr_html_()

In [ ]:
#Etapa 6: Integrando LangChain ao aplicativo

def find_restaurants(city_name: str, restaurant_kind: str, google_maps_api_key: str, open_ai_api_key: str):
    # Create the input text for the agent to find vegan restaurants in the specified city
    input_text = f"Encontre restaurantes {restaurant_kind} em {city_name}."

    # Invoke the agent executor with the input text and an empty chat history
    result = agent_executor.invoke({"input": input_text, "chat_history": []})

    # Extract the output from the result
    output = result['output']

    # This function should realistically fetch data based on the agent's output
    restaurants = get_restaurants_by_kind(google_maps_api_key, city_name, restaurant_kind)

    # Create a list of locations with the necessary details for mapping
    locations = [{'name': r['name'], 'lat': r['geometry']['location']['lat'], 'lng': r['geometry']['location']['lng']} for r in restaurants]

    # Create an HTML map using the locations data
    map_html = create_map(locations)

    # Return the agent's output and the generated HTML map
    return output, map_html

def generate_itinerary(city_name: str, duration: int, kind: str, google_maps_api_key: str, open_ai_api_key: str):
    # Create the input text for the agent to make an itinerary for the specified number of days in the city
    input_text = f"Faça um itinerário de {duration} dias em {city_name}, incluindo restaurantes {kind} e pontos de interesse."

    # Set up the chat history with the user's query and the agent's response
    chat_history = [
      {"role": "user", "content": f"Quais estabelecimentos {kind} eu posso visitar em {city_name}?"},
        {"role": "assistant", "content": find_restaurants(city_name, kind, google_maps_api_key, open_ai_api_key)[0]}
    ]

    # Invoke the agent executor with the input text and the updated chat history
    result = agent_executor.invoke({"input": input_text, "chat_history": chat_history})

    # Extract the output from the result
    output = result['output']

    # This function should realistically fetch data based on the agent's output
    restaurants = get_restaurants_by_kind(google_maps_api_key, city_name, kind)

    # Create a list of locations for the itinerary, limited by the duration
    locations = [{'name': r['name'], 'lat': r['geometry']['location']['lat'], 'lng': r['geometry']['location']['lng']} for r in restaurants[:duration]]

    # Create an HTML map with the route drawn using the locations data
    map_html = create_map(locations, draw_route=True)

    # Return the agent's output and the generated HTML map
    return output, map_html
    # Simulação da extração de coordenadas dos pontos do itinerário
    restaurants = get_restaurants_by_kind(google_maps_api_key, city_name, kind)
    locations = [{'name': r['name'], 'lat': r['geometry']['location']['lat'], 'lng': r['geometry']['location']['lng']} for r in restaurants[:duration]]

    map_html = create_map(locations)
    return output, map_html

In [ ]:
#Etapa 7: Construindo a Interface Gradio
import gradio as gr

# Defining Gradio components for the first tab
city_input_tab1 = gr.Textbox(label="Cidade", placeholder="Escreva o nome da cidade")
kind_tab1 = gr.Textbox(label="Tipo de restaurante", placeholder="Escreva um tipo de restaurante")
restaurants_output = gr.Textbox(label="Restaurantes")
restaurants_map = gr.HTML(label="Map")

# Defining Gradio components for the second tab
city_input_tab2 = gr.Textbox(label="Cidade", placeholder="Escreva o nome da cidade")
kind_tab2 = gr.Textbox(label="Tipo de restaurante", placeholder="Escreva um tipo de restaurante")
days_input = gr.Slider(label="Número de dias", minimum=1, maximum=10, step=1, value=5)
itinerary_output = gr.Textbox(label="Itinerário")
itinerary_map = gr.HTML(label="Map")

# Creating the Gradio interface with tabs
tab1 = gr.Interface(
    fn=lambda city_name, kind: find_restaurants(city_name, kind, google_maps_api_key, open_ai_api_key),
    inputs=[city_input_tab1, kind_tab1],
    outputs=[restaurants_output, restaurants_map],
    live=False
)

tab2 = gr.Interface(
    fn=lambda city_name, duration, kind: generate_itinerary(city_name, duration, kind, google_maps_api_key, open_ai_api_key),
    inputs=[city_input_tab2, days_input, kind_tab2],
    outputs=[itinerary_output, itinerary_map]
)

iface = gr.TabbedInterface([tab1, tab2], ["Encontre restaurantes", "Gerar itinerário"])

iface.launch(debug=True)

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://4b5a94f738e2713f3b.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)




> Entering new AgentExecutor chain...

Invoking: `find_restaurants_by_kind` with `{'city_name': 'São Paulo', 'kind': 'Italiano'}`


[{'name': 'Jardim de Napoli', 'address': 'R. Martinico Prado, 463 - Vila Buarque, São Paulo', 'lat': -23.5427105, 'lng': -46.655839}, {'name': 'Famiglia Mancini Trattoria', 'address': 'Rua Avanhandava, 81, Bela Vista - Centro Histórico de São Paulo, São Paulo', 'lat': -23.5503587, 'lng': -46.6450067}, {'name': 'Osteria Generale - Bela Vista', 'address': 'R. Dr. Fausto Ferraz, 163 - Bela Vista, São Paulo', 'lat': -23.565854, 'lng': -46.6466457}, {'name': 'Lellis Trattoria', 'address': 'R. Bela Cintra, 1849 - Jardins, São Paulo', 'lat': -23.5614601, 'lng': -46.6669749}, {'name': 'Lellis Trattoria', 'address': 'Alameda Campinas, 1615 - Jardim Paulista, São Paulo', 'lat': -23.5734754, 'lng': -46.66041269999999}, {'name': 'Pecorino Bar & Trattoria - Vila Nova Conceição', 'address': 'R. Domingos Leme, 687 - Vila Nova Conceição, São Paulo', 'lat': -23.5941001, 